In [4]:
!conda env list

# conda environments:
#
base                     /opt/conda



In [5]:
import torch
import numpy as np

In [6]:
def from_torch_to_numpy(tensor: torch.Tensor):
    return tensor.detach().cpu().numpy()

In [7]:

def energy_function(p_bits: torch.Tensor, weights: torch.Tensor, bias: torch.Tensor):
    return np.sum(np.multiply(from_torch_to_numpy(weights), from_torch_to_numpy(torch.outer(p_bits, p_bits)))) + torch.dot(p_bits, bias)


In [8]:
def calculate_gradient(p_bits: torch.Tensor, weights: torch.Tensor, bias: torch.Tensor):
    energy = energy_function(p_bits, weights, bias) 
    energy.backward(torch.ones(energy.shape))
    p_bits.retain_grad()
    return p_bits.grad

In [9]:
def activation_function(syn_input: torch.Tensor, beta: float):
    return torch.sign(torch.tanh(beta * (-syn_input)) - (torch.rand(1) * 2 - 1))


In [10]:
def find_energy_minimum(initial_beta, weights: torch.Tensor, bias: torch.Tensor, n: int):
    p_bits = torch.ones(n, requires_grad=True)
    prev_energy = energy_function(p_bits, weights, bias)
    beta = initial_beta
    ans = torch.ones(n, requires_grad=True) 
    last_change_index = 0
    for i in range(100):
        indices = torch.randperm(n)
        for index, value in enumerate(indices):
            new_beta = beta + 0.1
            iteration = n * i + index
            gradient = calculate_gradient(p_bits, weights, bias)
            syn_input = gradient[value]
            old_value = p_bits[value].item()
            with torch.no_grad():
                p_bits[value] = activation_function(syn_input, new_beta)
            new_energy = energy_function(p_bits, weights, bias)
            
            if torch.abs(new_energy) < torch.abs(prev_energy): 
                beta = new_beta
                prev_energy = new_energy
                ans = p_bits.clone()
    return ans.tolist()
 

## **Number partitioning task**

In [11]:
numbers = range(1, 32)
n = len(numbers)
weights = torch.zeros((n, n), dtype=torch.float32)
for i in range(n):
    for j in range(n):
        weights[i][j] = numbers[i] * numbers[j] 
bias = torch.zeros(n, dtype=torch.float32)

seed = 42
torch.manual_seed(seed)
result = find_energy_minimum(0.5, weights, bias, n)

if torch.dot(torch.tensor(numbers, dtype=torch.float32), torch.tensor(result)) != 0:
    print("The partitioning of this numbers is impossible")
else:
    print("First set of numbers: ")
    first_set_of_numbers = []
    second_set_of_numbers = []
    for index, el in enumerate(result):
        if el == 1.0:
            first_set_of_numbers.append(int(el * numbers[index]))
        else:
            second_set_of_numbers.append(int(-el * numbers[index]))    
    print(*first_set_of_numbers)
    print("Second set of numbers: ")
    print(*second_set_of_numbers)

First set of numbers: 
1 3 4 6 8 9 11 14 17 19 20 23 26 28 29 30
Second set of numbers: 
2 5 7 10 12 13 15 16 18 21 22 24 25 27 31


## **Graph partitioning task**

In [12]:
n = 8
pairs = {1: [2, 3, 0], 2: [1, 3, 0], 3: [1, 2, 0], 0: [1, 2, 3], 5: [4, 6, 7]
, 6: [4, 5, 7], 7: [4, 5, 6], 4: [5, 6, 7]}
# n = 6
# pairs = {1: [2, 0], 2: [1, 0], 0: [2, 1], 3: [4, 5], 4: [5, 3]
# , 5: [4, 3]}
# n = 4
# pairs = {1: [2, 3], 2: [1], 3: [1, 0], 0: [3]}

weights = torch.zeros((n, n), dtype=torch.float32)
free_term = 0 
for i in range(n):
    for j in range(n):
        if j in pairs[i]:
            weights[i][j] -= 1 / 2
            free_term += 1 / 2
        weights[i][j] += 1
weights[0][0] += free_term

bias = torch.zeros(n, dtype=torch.float32)
seed = 42
torch.manual_seed(seed)
result = find_energy_minimum(0.5, weights, bias, n)

first_set_of_vertices = []
second_set_of_vertices = []
for index, el in enumerate(result):
    if el == 1.0:
        first_set_of_vertices.append(int(el * numbers[index]) - 1)
    else:
        second_set_of_vertices.append(int(-el * numbers[index]) - 1)    

print("First set of vertices: ")
print(*first_set_of_vertices)
print("Second set of vertices: ")
print(*second_set_of_vertices)


First set of vertices: 
0 1 2 3
Second set of vertices: 
4 5 6 7


## Computation time difference for Number Paritioning

In [13]:
import time
import matplotlib.pyplot as plt
import itertools

In [14]:
def complete_search(numbers):
    total_sum = sum(numbers)
    num_elements = len(numbers)
    
    best_partition = None
    min_partition_diff = float('inf')
    for i in range(1, num_elements // 2 + 1):
        for subset in itertools.combinations(numbers, i):
            subset_sum = sum(subset)
            complement_sum = total_sum - subset_sum
            partition_diff = abs(subset_sum - complement_sum)
            
            if partition_diff < min_partition_diff:
                min_partition_diff = partition_diff
                best_partition = (list(subset), [x for x in numbers if x not in subset])
    
    return best_partition

In [19]:
test_cases = [30, 31]

execution_times_find_energy_minimum = []
execution_times_complete_search = []


for test in test_cases:
    numbers = range(1, test + 1)
    complete_search_time_start = time.time()
    complete_search(numbers)
    complete_search_time_end = time.time()
    execution_times_complete_search.append(complete_search_time_end - complete_search_time_start)
    find_energy_minimum_time_start = time.time()
    n = len(numbers)
    weights = torch.zeros((n, n), dtype=torch.float32)
    for i in range(n):
        for j in range(n):
            weights[i][j] = numbers[i] * numbers[j] 
    bias = torch.zeros(n, dtype=torch.float32)

    result = find_energy_minimum(0.5, weights, bias, n)
    find_energy_minimum_time_end = time.time()
    execution_times_find_energy_minimum.append(find_energy_minimum_time_end - find_energy_minimum_time_start)



In [20]:
print(execution_times_complete_search)
print(execution_times_find_energy_minimum)

[166.19825863838196, 294.98395586013794]
[1.0487761497497559, 1.011204719543457]


In [17]:
import pandas as pd
import altair as alt

data = pd.DataFrame({
    'Input Cases': test_cases,
    'Complete Search Execution Time': execution_times_complete_search,
    'Our Approach Execution Time': execution_times_find_energy_minimum
})

chart = alt.Chart(data).mark_line(point=True).encode(
    x='Input Cases',
    y='Complete Search Execution Time:Q',
    color=alt.value('blue'),
    tooltip=['Input Cases', 'Complete Search Execution Time']
).properties(
    title='Computational Difference Between Complete Search and Our Approach',
    width=500
) + alt.Chart(data).mark_line(point=True).encode(
    x='Input Cases',
    y='Our Approach Execution Time',
    color=alt.value('red'),
    tooltip=['Input Cases', 'Our Approach Execution Time']
)

chart.interactive()

alt.LayerChart(...)